## TP02 

In [ ]:
import os 
from math import ceil

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.colors as colors
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

import netCDF4 as nc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from geographiclib.geodesic import Geodesic

geod = Geodesic.WGS84 
RADIUS_EARTH = 6371.009

def define_params_recta(p1, p2):
    m = (p2[1] - p1[1])/(p2[0] - p1[0])
    b = p1[1] - m * p1[0]
    return m, b

def lineal(x, m, b):
    return m * x + b


def euc_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) 


def geoline_way_points(p1, p2, ds=10e3):
    """
    ds = distance steps
    """
    lat1, lon1 = p1
    lat2, lon2 = p2
    lat = []
    lon = []
    d = []
    geoline = geod.InverseLine(lat1, lon1, lat2, lon2) 
    n = int(ceil(geoline.s13 / ds))
    for i in range(n + 1):
        s = min(ds * i, geoline.s13)
        g = geoline.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        d.append(g["s12"])
        lat.append(g["lat2"])
        lon.append(g["lon2"])
    return pd.DataFrame({"distance":d, "lat":lat, "lon":lon})


def np_geodistance_sphere(lat1, lon1, lat2, lon2):
    dlon = lon2 - lon1
    slat1, clat1 = np.sin(lat1), np.cos(lat1)
    slat2, clat2 = np.sin(lat2), np.cos(lat2)
    sdlon, cdlon = np.sin(dlon), np.cos(dlon)
    arg1 = np.sqrt((clat2 * sdlon)**2 + (clat1 * slat2 - slat1 * clat2 * cdlon)**2)
    arg2 = slat1 * slat2 + clat1 * clat2 * cdlon
    return RADIUS_EARTH * np.arctan2(arg1, arg2)

def degree_to_radians(n):
    return n * np.pi / 180


In [ ]:
DATADIR = ".\data"
nf = "AQUA_MODIS.20191127T175000.L2.OC.nc" 

In [ ]:
data = nc.Dataset(os.path.join(DATADIR, nf), mode='r', mmap=False)

**1. Obtenga una imagen L2 diaria en formato netcdf correspondiente al día 27 de nov de 2019. Escriba el nombre del archivo y ábralo en el SNAP. Mencione cuántas bandas contiene el archivo en la carpeta BANDS.**


```
data.groups["sensor_band_parameters"].variables["wavelength"].shape
```

In [ ]:
data.groups["sensor_band_parameters"].variables["wavelength"].shape

**2. De la carpeta de metadatos del archivo .nc extraiga la hora de inicio y fin de la imagen.**


In [ ]:
print("Horario de inicio: ", data.time_coverage_start)
print("Horario de fin:    ", data.time_coverage_end)

**3. Despliegue la imagen de Clor-a (producto geofísico que estima la biomasa de fitoplancton y es derivado de la reflectancia que llega al sensor satelital). Aplique una paleta de colores y modifique el rango del mínimo y máxima para que sean identificables los contrastes.**



In [ ]:
#data = nc.Dataset(os.path.join(DATADIR, nf), mode='r', mmap=False)
geodata  = data.groups['geophysical_data']
navigation = data.groups['navigation_data']
lon = navigation.variables['longitude'][:]
lat = navigation.variables['latitude'][:]
clorofila = data.groups['geophysical_data'].variables['chlor_a'][:]
# Cierro el dataset
#data.close()

Para graficar vamos a necesitar dos cosas:  
1. Transformarlos datos a una escala logaritmica para visualizar mejor las diferencias.  
2. Crear una escala de colores que me permita seguir esta escala logaritmica. 

Para hacer el segundo paso en python vamos a crear una función:

```
def get_chlorophyll_color():
    lowchlorophyll = plt.cm.Reds(np.linspace(0.3, 0.8, 7))
    highchlorophyll = plt.cm.Greens(np.linspace(0.4, 0.8, 5))
    joincolors = np.vstack((lowchlorophyll, highchlorophyll))
    return colors.LinearSegmentedColormap.from_list('chlorophyll_color', joincolors)
```

La función usa matplotlib.pyplot (as plt) y matplotlib.colors (as colors).  


Una vez definida nuestra función color, el resto es aplicar la escala logaritmica y graficar. Cuando grafiquemos, el cmap debe ser igual a clorophyl_color().

In [ ]:
def get_chlorophyll_color():
    lowchlorophyll = plt.cm.Reds(np.linspace(0.3, 0.8, 7))
    highchlorophyll = plt.cm.Greens(np.linspace(0.4, 0.8, 5))
    joincolors = np.vstack((lowchlorophyll, highchlorophyll))
    chlorophyll_color = colors.LinearSegmentedColormap.from_list('chlorophyll_color', joincolors)
    return chlorophyll_color

In [ ]:
chlog_10 = np.log10(clorofila)
levels = [.001, .1, .2, .3, .5, .8, 1, 5, 10, 85]
log10_levels = np.log10(levels)# Tick mark positions
# ticks = [-3, -1,  0, 0.69, 1, 1.93]
# nameticks = [0.001, 0.1, 1, 5, 10, 85]

In [ ]:
#GRAFICO 
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(), zorder=0)
ax.set_title("Clorofila Superficial \n 2019-11-27")

CS = ax.contourf(lon, 
                 lat, 
                 chlog_10, 
                 levels=log10_levels, 
                 cmap=get_chlorophyll_color(),
                 vmin=np.log10(clorofila.min()), 
                 vmax=np.log10(clorofila.max()), 
                 extend='both',
                 zorder=0)

cbar = fig.colorbar(CS, 
                    ticks=log10_levels, # ticker.LogLocator(base=10.0, numticks=50), 
                    label='Concentracion [$\\frac{mg}{m^3}$]', 
                    shrink=0.5, 
                    extend='both')

cbar.ax.set_yticklabels(levels)

ax.set_facecolor("silver")
ax.add_feature(cfeature.LAND, facecolor="white", zorder=1)
#ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), color="white", draw_labels=True, linestyle='--', linewidth=1, zorder=0)
gl.ylabels_right = False
gl.xlabels_top = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.xlabel_style = {'size':12}
gl.ylabel_style = {'size':12}

plt.show()

**3. Grafique los datos de Clor-a a lo largo de una transecta sobre la imagen**

Para definir una recta vamos a tomar dos puntos en un  sistema (lon, lat) uniendolos en una recta. Para esto vamos a usar dos funciones:  

**geoline_way_points**  
Función que me va a dar todos los puntos intermedios en (lat, lon) entre las puntos que definen la transecta y la distancia al punto de origen (tambien se podria agregar el valor azimutal). Para esto se usa la libreria `geographiclib` (libreria de fondo de geopy).

```
from geographiclib.geodesic import Geodesic

geod = Geodesic.WGS84 

def geoline_way_points(p1, p2, ds=10e3):
    """
    ds = distance steps
    """
    lat1, lon1 = p1
    lat2, lon2 = p2
    lat = []
    lon = []
    d = []
    geoline = geod.InverseLine(lat1, lon1, lat2, lon2) 
    n = int(ceil(geoline.s13 / ds))
    for i in range(n + 1):
        s = min(ds * i, geoline.s13)
        g = geoline.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        d.append(g["s12"])
        lat.append(g["lat2"])
        lon.append(g["lon2"])
    return pd.DataFrame({"distance":d, "lat":lat, "lon":lon})
```
**np_geodistance_sphere**  
Función que calcula la distancia entre dos puntos asumiendo una geodesica en una esfera del radio de la tierra. 
Esto se hace para acelerar los calculos a la hora de estimar distancias, es una aproximación y deberia usarse la funcion geopy.distance pero para los usos de ahora nos sirve. 

```
RADIUS_EARTH = 6371.009

def np_geodistance_sphere(lat1, lon1, lat2, lon2):
    dlon = lon2 - lon1
    slat1, clat1 = np.sin(lat1), np.cos(lat1)
    slat2, clat2 = np.sin(lat2), np.cos(lat2)
    sdlon, cdlon = np.sin(dlon), np.cos(dlon)
    arg1 = np.sqrt((clat2 * sdlon)**2 + (clat1 * slat2 - slat1 * clat2 * cdlon)**2)
    arg2 = slat1 * slat2 + clat1 * clat2 * cdlon
    return RADIUS_EARTH * np.arctan2(arg1, arg2)
```

Una vez esto, y definidos dos puntos, podemos graficar nuestra recta en el mapa: 

In [ ]:
## Tomamos los puntos
p1 = (-58, -43)
p2 = (-53, -41)
invp1 = (p1[1], p1[0])
invp2 = (p2[1], p2[0])

rinvp1 = degree_to_radians(np.asarray(invp1))

In [ ]:
dfway = geoline_way_points(invp1, invp2)
dfway["distance"] = dfway["distance"]/1000

In [ ]:
#GRAFICO 
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(), zorder=0)
ax.set_title("Clorofila Superficial \n 2019-11-27")

plt.plot([p1[0], p2[0]], [p1[1], p2[1]], "bo--", label=f"Transecta:\n    p1: {p1}\n    p2: {p2}")
# plt.plot(dfway.lon.tolist(), dfway.lat.tolist(), "co", linewidth=0.1, label="Puntos de la transecta")
CS = ax.contourf(lon, 
                 lat, 
                 chlog_10, 
                 levels=log10_levels, 
                 cmap=get_chlorophyll_color(),
                 vmin=np.log10(clorofila.min()), 
                 vmax=np.log10(clorofila.max()), 
                 extend='both',
                 zorder=0)

cbar = fig.colorbar(CS, 
                    ticks=log10_levels, # ticker.LogLocator(base=10.0, numticks=50), 
                    label='Concentracion [$\\frac{mg}{m^3}$]', 
                    shrink=0.5, 
                    extend='both')

cbar.ax.set_yticklabels(levels)

ax.set_facecolor("silver")
ax.add_feature(cfeature.LAND, facecolor="white", zorder=1)
#ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), color="white", draw_labels=True, linestyle='--', linewidth=1, zorder=0)
gl.ylabels_right = False
gl.xlabels_top = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.xlabel_style = {'size':12}
gl.ylabel_style = {'size':12}

plt.legend()
plt.show()

La pregunta ahora es:  **¿Como tomamos los  valores de clorofila?**

Los datos de todo el sistema vienen dados en dos dimensiones, "linea" y "pixel". Existe una transformación que me lleva a cada punto en mi sistema (linea, pixel) a un sistema (lon, lat). Es decir, 

$$lat = F(linea, pixel)$$
y
$$lon = G(linea, pixel) $$

y esto, a su vez, nos deberia poder permitir definir las funciones:
$$linea = f(lat, lon)$$ y $$ pixel = g(lat, lon)$$

Vamos asumir que cada "punto" tiene dos errores asociados un delta entre lineas (el cual asumirmos que cambia a lo largo de la linea, pero es constante entre lineas) y un delta entre pixeles de una misma linea (el cual vamos asumir constante por linea). Entonces, vamos a mirar que "areas" atravieza la la recta, tomamos esos puntos de clorofila y lo asociamos a los puntos de la transecta (el delta entre los puntos de la transecta lo definimos nosotros, pero debe ser mayor al radio maximo del error.  

In [ ]:
dlineas = []
dpixel = []
for ix in range(lon.shape[0]-1):
    LON1 = lon[ix, :]
    LON2 = lon[ix + 1, :]
    LAT1 = lat[ix, :]
    LAT2 = lat[ix + 1, :]
    D = np_geodistance_sphere(
            degree_to_radians(LAT1), degree_to_radians(LON1), 
            degree_to_radians(LAT2), degree_to_radians(LON2)
        )
    dlineas.append(D)
elineas = np.mean(np.asarray(dlineas), axis=0)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Distancia entre lineas por pixel")
plt.xlabel("N° pixel")
plt.ylabel("Distancia lineas[Km]")
for l in dlineas:
    plt.plot(l, "--", color="grey", alpha=.01)
plt.grid()
plt.plot(elineas, color="c", linewidth=3, label="distancia entre lineas promedio")

En la figura se puede ver en celeste el error que se hará uso. Notar como hay una menor cantidad de lineas donde el error queda subestimado (el gris mas claro indica que son "menos lineas"). Sigue sin ser perfecto, pero es una  buena aproximación.

In [ ]:
dpixel = []
for ix in range(lon.shape[1]-1):
    LON1 = lon[:, ix]
    LON2 = lon[:, ix + 1]
    LAT1 = lat[:, ix]
    LAT2 = lat[:, ix + 1]
    D = np_geodistance_sphere(
            degree_to_radians(LAT1), degree_to_radians(LON1), 
            degree_to_radians(LAT2), degree_to_radians(LON2)
        )
    dpixel.append(D)
    
epixel = np.mean(np.asarray(dpixel), axis=0)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Distancia entre pixeles por linea")
plt.xlabel("N° linea")
plt.ylabel("Distancia pixeles[Km]")

for l in dpixel:
    plt.plot(l, "--", color="grey", alpha=.01)
plt.plot(epixel, color="c", linewidth=4, label="Distancia promedio pixeles")

plt.legend()
plt.grid()
plt.show()

Nuevamente, el erro en algunos casos va a quedar subestimado y en otros sobre estimado. Pero nos sirve como aproximación.

PEEEEROO La ecuación de una elipse viene  dada por, donde p1 y p2 representa el punto central de mi elipse con x, y variables parametrizadas del contorno de la elipse. 

$$\frac{(x - p_1)^2}{a^2} + \frac{(y - p_2)^2}{b^2} = 1$$

si el punto de la recta esta en el interior de la parametrización, se toma la clorofila. Para simplificar aun más problema, no vamos a tomar una elipse sino una esfera de radio = max(error_pixel, error_linea). Basta entonces que la distancia entre los puntos sea menor a ese radio para tomarlo como punto de la recta.

In [ ]:
error = []
for ep in epixel:
    auxer = []
    for el in elineas:
        auxer.append(ep if ep > el else el)
    error.append(auxer)
    
error = np.asarray(error)

In [ ]:
LAT = dfway.lat.to_numpy()
LON = dfway.lon.to_numpy()
WAYLAT = degree_to_radians(LAT)
WAYLON = degree_to_radians(LON)
RLAT = degree_to_radians(lat)
RLON = degree_to_radians(lon)

In [ ]:
clomean = []
clostd = []
for i in range(WAYLAT.shape[0]):
    cond = (np_geodistance_sphere(RLAT, RLON, WAYLAT[i], WAYLON[i]) < error)
    clo = clorofila[cond]
    clomean.append(clo.mean())
    clostd.append(clo.std())

dfway["clorofila"] = [np.nan if np.ma.is_masked(c) else c for c in clomean]
dfway["std"]= [np.nan if np.ma.is_masked(c) else c for c in clostd]

In [ ]:
dfway["clolog10"] = dfway.clorofila.apply(lambda x: np.log10(x))
dfway["stdlog10"]= dfway.apply(lambda row: row["std"]/row["clorofila"], axis=1) 

In [ ]:
plt.title("Clorofila en la transecta")
plt.ylabel("Clorofila")
plt.xlabel("distancia trasecta[Km]")
# plt.yticks(ticks=log10_levels, labels=levels)
plt.plot(dfway.distance.to_numpy(), dfway.clorofila.to_numpy(), "co--")

plt.fill_between(dfway.distance.to_numpy(), 
                dfway.clorofila.to_numpy() - dfway["std"].to_numpy(), 
                dfway.clorofila.to_numpy() + dfway["std"].to_numpy(), 
                color="c", 
                alpha=0.3,
                label="error")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.title("Clorofila(log10) en la transecta")
plt.ylabel("Clorofila[log10]")
plt.xlabel("distancia trasecta[Km]")
# plt.yticks(ticks=log10_levels, labels=levels)
plt.plot(dfway.distance.to_numpy(), dfway.clolog10.to_numpy(), "co--")

plt.fill_between(dfway.distance.to_numpy(), 
                dfway.clolog10.to_numpy() - dfway["stdlog10"].to_numpy(), 
                dfway.clolog10.to_numpy() + dfway["stdlog10"].to_numpy(), 
                color="c", 
                alpha=0.3,
                label="error")
plt.legend()
plt.grid()
plt.show()

In [ ]:
print("Tabla 1: Transecta y sus valores")
dfway